In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time

In [2]:
# Scrape website
# html=browser.html

# Find all the car listings on the page
# car_list=car_soup.find_all('div', class_='vehicle-card')

# Testing scraping for the first listing on the site
# car_model=car_soup.find('h2', class_='title')
# car_status=car_soup.find('p', class_='stock-type')
# car_price=car_soup.find('span', class_='primary-price')
# car_dealer=car_soup.find('div', class_='dealer-name')
# dealer_city=car_soup.find('div', class_='miles-from')

# print(car_model, car_price, car_dealer, car_status, dealer_city)


In [3]:
# Testing to make sure text that is stripped is correct
#model=car_model.get_text(strip=True)
#status=car_status.get_text(strip=True)
#price=car_price.get_text(strip=True)
#dealer=car_dealer.get_text(strip=True)
#city=dealer_city.get_text(strip=True)

#print(model, status, price, dealer, city)

In [4]:
# Set up Splinter
browser = Browser('chrome')

# Base URL and initial page
base_url = 'https://www.cars.com/shopping/results/?makes[]=nissan&maximum_distance=150&models[]=nissan-rogue&page={}&page_size=100&stock_type=all&zip=61606'
page_number = 1
url = base_url.format(page_number)

# Create lists to store details
models = []
statuses = []
mileages = []
prices = []
dealers = []
cities = []
urls = []  # List to store URLs

# Set to store unique identifiers of scraped cars
scraped_car_ids = set()

# Function to scrape current page
def scrape_page(url):
    browser.visit(url)
    time.sleep(5)  # Adjust as necessary
    
    html = browser.html
    car_soup = soup(html, 'html.parser')
    car_list = car_soup.find_all('div', class_='vehicle-card')

    if not car_list:
        print("No more car listings found. Exiting.")
        return False

    for car in car_list:
        # Extract car model
        car_model = car.find('h2', class_='title')
        model = str(car_model.get_text(strip=True)) if car_model else 'N/A'

        # Extract car mileage
        car_mileage = car.find('div', class_='mileage')
        mileage = str(car_mileage.get_text(strip=True)) if car_mileage else 'N/A'

        # Extract car status
        car_status = car.find('p', class_='stock-type')
        status = str(car_status.get_text(strip=True)) if car_status else 'N/A'

        # Extract car price
        car_price = car.find('span', class_='primary-price')
        price = str(car_price.get_text(strip=True)) if car_price else 'N/A'

        # Extract dealer name
        car_dealer_div = car.find('div', class_='dealer-name')
        if car_dealer_div:
            car_dealer = car_dealer_div.find('strong')
            dealer = str(car_dealer.get_text(strip=True)) if car_dealer else 'N/A'
        else:
            dealer = 'N/A'

        # Extract dealer city
        dealer_city = car.find('div', class_='miles-from')
        city = str(dealer_city.get_text(strip=True)) if dealer_city else 'N/A'

        # Extract car URL
        car_url = car.find('a', class_='vehicle-card-link')['href'] if car.find('a', class_='vehicle-card-link') else 'N/A'

        # Check if this car has already been scraped
        if car_url in scraped_car_ids:
            continue  # Skip if already scraped
        
        # Add car URL to set of scraped car IDs
        scraped_car_ids.add(car_url)

        # Append details to lists
        models.append(model)
        statuses.append(status)
        prices.append(price)
        dealers.append(dealer)
        cities.append(city)
        mileages.append(mileage)
        urls.append(car_url)  # Append URL to list

    return True

# Counter to track consecutive pages with no new data
no_new_data_count = 0
max_no_new_data_allowed = 3  # Adjust as necessary

# Timeout settings
timeout_seconds = 240  # 4 minutes timeout
start_time = time.time()

# Loop through pages
while True:
    if time.time() - start_time > timeout_seconds:
        print(f"Timeout ({timeout_seconds} seconds) exceeded. Exiting.")
        break
    
    if not scrape_page(url):
        # Increment no_new_data_count if no new data found
        no_new_data_count += 1
        if no_new_data_count >= max_no_new_data_allowed:
            print(f"No new data found for {max_no_new_data_allowed} consecutive pages. Exiting.")
            break
    else:
        # Reset no_new_data_count if new data found
        no_new_data_count = 0
    
    # Increment page number and update URL
    page_number += 1
    url = base_url.format(page_number)
    print(f"Scraping page {page_number}...")

# Print number of cars scraped
print(f"Number of unique cars scraped: {len(scraped_car_ids)}")

Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Timeout (240 seconds) exceeded. Exiting.
Number of unique cars scraped: 2820


In [5]:
# Create df
data={
    'Model': models,
    'Mileage (mi)': mileages,
    'Status': statuses,
    'Price (USD)': prices,
    'Dealer': dealers,
    'City': cities ,
    'URL': urls
    }
car_df= pd.DataFrame(data)

In [6]:
car_df

,Model,Mileage (mi),Status,Price (USD),Dealer,City,URL
0,2018 Nissan Rogue SV,"38,067 mi.",Used,"$19,995",DriveTime of Peoria,"Peoria, IL (6 mi.)",/vehicledetail/3021e03c-366f-49d4-920a-1f16c77...
1,2021 Nissan Rogue S,"21,173 mi.",Used,"$22,697",Bill Walsh Streator,"Streator, IL (52 mi.)",/vehicledetail/9d59605b-05a8-4614-86f2-03d35f2...
2,2024 Nissan Rogue S,N/A,New,"$30,425","JP Motors, Inc.","Peru, IL (52 mi.)",/vehicledetail/91c4b117-1f23-4cbb-b905-5425e23...
3,2020 Nissan Rogue S,"22,342 mi.",Used,"$22,998",CarMax Austin North - Offering Express Pickup ...,N/A,/vehicledetail/6a542586-d71e-4a8b-aec4-0afbc4c...
4,2022 Nissan Rogue SV,"8,307 mi.",Used,"$26,998",CarMax Newark - Offering Express Pickup and Ho...,N/A,/vehicledetail/45100b6d-9b80-460e-8bbc-615a0e9...
...,...,...,...,...,...,...,...
2815,2024 Nissan Rogue SV,N/A,New,Not Priced,Al Piemonte Nissan,"Melrose Park, IL (124 mi.)",/vehicledetail/d5a1132a-f15e-41a0-bf0a-39b8e43...
2816,2023 Nissan Rogue SL,"4,000 mi.",Nissan Certified,"$33,500",Al Piemonte Nissan,"Melrose Park, IL (124 mi.)",/vehicledetail/d15a4773-f133-44b1-8b29-53b7a69...
2817,2024 Nissan Rogue SV,N/A,New,Not Priced,Al Piemonte Nissan,"Melrose Park, IL (124 mi.)",/vehicledetail/a3c0fad7-f7de-42de-b60c-704db8d...
2818,2017 Nissan Rogue SV,"44,300 mi.",Nissan Certified,"$18,000",Al Piemonte Nissan,"Melrose Park, IL (124 mi.)",/vehicledetail/e8cf5334-8998-4591-9267-8dafc87...


In [7]:
# Checking to make sure df is formatted correctly
# print(car_df.to_string())

In [8]:
# Close the browser
browser.quit()

In [9]:
# Display DF
print("Original DataFrame:")
print(car_df)
print()

Original DataFrame:
                     Model Mileage (mi)            Status Price (USD)  \
0     2018 Nissan Rogue SV   38,067 mi.              Used     $19,995   
1      2021 Nissan Rogue S   21,173 mi.              Used     $22,697   
2      2024 Nissan Rogue S          N/A               New     $30,425   
3      2020 Nissan Rogue S   22,342 mi.              Used     $22,998   
4     2022 Nissan Rogue SV    8,307 mi.              Used     $26,998   
...                    ...          ...               ...         ...   
2815  2024 Nissan Rogue SV          N/A               New  Not Priced   
2816  2023 Nissan Rogue SL    4,000 mi.  Nissan Certified     $33,500   
2817  2024 Nissan Rogue SV          N/A               New  Not Priced   
2818  2017 Nissan Rogue SV   44,300 mi.  Nissan Certified     $18,000   
2819   2023 Nissan Rogue S   15,084 mi.  Nissan Certified     $25,000   

                                                 Dealer  \
0                                   DriveTim

In [10]:
# Check for missing values:
missing_values= car_df.isnull().sum()
print("Missing values:")
print(missing_values)
print()

Missing values:
Model           0
Mileage (mi)    0
Status          0
Price (USD)     0
Dealer          0
City            0
URL             0
dtype: int64



In [11]:
# Split 'Model' column into 'Year', 'Manufacturer', 'Model', 'Trim'
car_df[['Year', 'Manufacturer', 'Model', 'Trim']] = car_df['Model'].str.split(' ', 3, expand=True)

# Replace 'Not Priced' with NaN and convert 'Price' column to numeric
car_df['Price (USD)'] = pd.to_numeric(car_df['Price (USD)'].replace('Not Priced', pd.NA).str.replace('[\$,]', '', regex=True), errors='coerce')

# Clean Mileage column
car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].replace('N/A', pd.NA).str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')

# Reorder columns as required
car_df = car_df[['Year', 'Manufacturer', 'Model', 'Trim', 'Mileage (mi)', 'Status', 'Price (USD)', 'Dealer', 'City', 'URL']]

# Display the updated DataFrame
print("Final DataFrame:")
print(car_df)

Final DataFrame:
      Year Manufacturer  Model Trim  Mileage (mi)            Status  \
0     2018       Nissan  Rogue   SV       38067.0              Used   
1     2021       Nissan  Rogue    S       21173.0              Used   
2     2024       Nissan  Rogue    S           NaN               New   
3     2020       Nissan  Rogue    S       22342.0              Used   
4     2022       Nissan  Rogue   SV        8307.0              Used   
...    ...          ...    ...  ...           ...               ...   
2815  2024       Nissan  Rogue   SV           NaN               New   
2816  2023       Nissan  Rogue   SL        4000.0  Nissan Certified   
2817  2024       Nissan  Rogue   SV           NaN               New   
2818  2017       Nissan  Rogue   SV       44300.0  Nissan Certified   
2819  2023       Nissan  Rogue    S       15084.0  Nissan Certified   

      Price (USD)                                             Dealer  \
0         19995.0                                DriveTime

C:\Users\awsom\AppData\Local\Temp\ipykernel_18684\2381806475.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].replace('N/A', pd.NA).str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')


In [13]:
car_df

,Year,Manufacturer,Model,Trim,Mileage (mi),Status,Price (USD),Dealer,City,URL
0,2018,Nissan,Rogue,SV,38067.0,Used,19995.0,DriveTime of Peoria,"Peoria, IL (6 mi.)",/vehicledetail/3021e03c-366f-49d4-920a-1f16c77...
1,2021,Nissan,Rogue,S,21173.0,Used,22697.0,Bill Walsh Streator,"Streator, IL (52 mi.)",/vehicledetail/9d59605b-05a8-4614-86f2-03d35f2...
2,2024,Nissan,Rogue,S,NaN,New,30425.0,"JP Motors, Inc.","Peru, IL (52 mi.)",/vehicledetail/91c4b117-1f23-4cbb-b905-5425e23...
3,2020,Nissan,Rogue,S,22342.0,Used,22998.0,CarMax Austin North - Offering Express Pickup ...,N/A,/vehicledetail/6a542586-d71e-4a8b-aec4-0afbc4c...
4,2022,Nissan,Rogue,SV,8307.0,Used,26998.0,CarMax Newark - Offering Express Pickup and Ho...,N/A,/vehicledetail/45100b6d-9b80-460e-8bbc-615a0e9...
...,...,...,...,...,...,...,...,...,...,...
2815,2024,Nissan,Rogue,SV,NaN,New,NaN,Al Piemonte Nissan,"Melrose Park, IL (124 mi.)",/vehicledetail/d5a1132a-f15e-41a0-bf0a-39b8e43...
2816,2023,Nissan,Rogue,SL,4000.0,Nissan Certified,33500.0,Al Piemonte Nissan,"Melrose Park, IL (124 mi.)",/vehicledetail/d15a4773-f133-44b1-8b29-53b7a69...
2817,2024,Nissan,Rogue,SV,NaN,New,NaN,Al Piemonte Nissan,"Melrose Park, IL (124 mi.)",/vehicledetail/a3c0fad7-f7de-42de-b60c-704db8d...
2818,2017,Nissan,Rogue,SV,44300.0,Nissan Certified,18000.0,Al Piemonte Nissan,"Melrose Park, IL (124 mi.)",/vehicledetail/e8cf5334-8998-4591-9267-8dafc87...


In [10]:
#print(car_df.to_string())

In [14]:
car_df.to_excel("Rogue.xlsx")